# 야구선수 연봉 예측

### index
1. 데이터 수집
2. 데이터 전처리
3. 데이터 매핑
4. 예측 모델 훈련
5. 최동원 선수 연봉 예측

# 1. 데이터 수집

## 투수 기록 데이터
- 1983년 ~ 1988년의 투수 데이터 (최동원 선수의 롯데시절)
- 2015년 ~ 2020년의 투수 데이터
- 선발/마무리 데이터 필요
    - 투수 데이터 URL (1983년 ~ 1988년) : http://www.kbreport.com/history/pitcher/main
            - 해당 URL에서는 1982년 ~ 2012년까지의 데이터가 존재한다.
        - 2015년 ~ 2020년 데이터를 확인하려면 아래 URL 활용
    - 투수 데이터 URL (1983년 ~ 1988년) : http://www.kbreport.com/leader/pitcher/main

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm import tqdm
from time import sleep


### 1. KBReport

#### 데이터 파싱 (1983 ~ 1988)

In [2]:
url = 'http://www.kbreport.com/history/pitcher/main'

driver = webdriver.Chrome()

driver.get(url)

In [3]:
# 연도 선택 토글에 들어있는 연도
years = driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select")
years_list = years.text.split()

years_list

['2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992',
 '1991',
 '1990',
 '1989',
 '1988',
 '1987',
 '1986',
 '1985',
 '1984',
 '1983',
 '1982']

- 필요한 연도는 1983 ~ 1988이므로 years_list에서 해당되지 않는 연도는 없애주자.

In [4]:
years_list = years_list[-7:-1]

years_list

['1988', '1987', '1986', '1985', '1984', '1983']

- years_list를 반복문을 돌면서 데이터 파싱

In [5]:
pitcher_data = []

for year in tqdm(years_list) :
    # 연도 선택 토글
    year_toggle = Select(driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select"))    
    year_toggle.select_by_value(str(year))
    # "결과" 선택 버튼
    result_btn = driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-right > button")
    result_btn.click()
    sleep(0.1)

    # Page Row를 100개로 맞추기
    page_row = Select(driver.find_element(By.CSS_SELECTOR, "#resultListDiv > div.page-row-box > select"))
    page_row.select_by_value(str(100))
    sleep(1.0)

    # 선수정보 가져오기
    pitcher_count = len(driver.find_element(By.CSS_SELECTOR, "#resultListDiv > table > tbody").find_elements(By.TAG_NAME, "tr"))

    for i in range(2, pitcher_count+1) :  # 가장 처음 tr 태그는 컬럼명이니 제외
        pitcher_info = driver.find_element(By.CSS_SELECTOR, f"#resultListDiv > table > tbody > tr:nth-child({i})")

        pitcher_info_list = pitcher_info.text.split()
        pitcher_info_list.append(year)
        pitcher_data.append(pitcher_info_list)

    print(pitcher_info_list)

 17%|█▋        | 1/6 [00:03<00:17,  3.54s/it]

['92', '정삼흠', 'MBC', '5', '16', '0', '27', '20', '139.7', '2.77', '3.93', '1.29', '0.297', '67.60', '5.67', '-1.36', '5.49', '5.39', '-1.39', '1988']


 33%|███▎      | 2/6 [00:06<00:13,  3.34s/it]

['88', '김강익', 'OB', '0', '2', '0', '12', '5', '37.0', '0.97', '4.38', '1.70', '0.274', '74.70', '5.11', '-0.47', '6.37', '6.09', '-1.12', '1987']


 50%|█████     | 3/6 [00:10<00:10,  3.44s/it]

...

['69', '오문현', '미/청', '4', '4', '0', '23', '7', '75.3', '1.91', '2.75', '2.27', '0.309', '74.70', '6.33', '-1.58', '6.63', '6.40', '-2.60', '1985']


 83%|████████▎ | 5/6 [00:17<00:03,  3.40s/it]

['59', '정성만', '삼미', '8', '10', '1', '29', '6', '150.0', '4.02', '4.98', '1.20', '0.267', '72.20', '4.86', '-0.82', '4.73', '4.70', '-1.56', '1984']


100%|██████████| 6/6 [00:20<00:00,  3.34s/it]

['55', '강철원', 'OB', '4', '2', '0', '17', '6', '62.7', '3.02', '3.30', '1.72', '0.288', '68.10', '6.32', '-1.03', '5.42', '5.33', '-1.26', '1983']


In [6]:
len(pitcher_data)

452

- pitcher_data를 value로 하는 데이터프레임 정의
- 컬럼 생성 (1983년도 기준)

In [7]:
# 컬럼 생성 (1983년도 기준)
columns = driver.find_element(By.CSS_SELECTOR, "#resultListDiv > table > tbody > tr:nth-child(1)")

report_columns = columns.text.split()

report_columns

['#',
 '선수명',
 '팀명',
 '승',
 '패',
 '세',
 '경기',
 '선발',
 '이닝',
 '삼진/9',
 '볼넷/9',
 '홈런/9',
 'BABIP',
 'LOB%',
 'ERA',
 'RA9-WAR',
 'FIP',
 'kFIP',
 'WAR']

In [8]:
driver.close()

- report_columns에는 year 정보가 없으니 year 추가

In [9]:
report_columns.append('year')

- 1983 ~ 1988 투수 정보 데이터프레임 생성

In [10]:
pitcher_df_old = pd.DataFrame(data=pitcher_data, columns=report_columns)

pitcher_df_old.head()

,#,선수명,팀명,승,패,세,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,year
0,1,선동열,해태,16,5,10,31,12,178.3,10.09,1.77,0.15,0.269,83.90,1.21,9.55,0.63,1.26,12.18,1988
1,2,윤학길,롯데,18,10,3,35,25,234.0,4.73,2.04,0.35,0.307,73.10,3.15,5.20,2.64,2.73,6.62,1988
2,3,최동원,롯데,7,3,3,16,4,83.3,8.96,2.59,0.43,0.317,80.90,2.05,2.51,1.48,2.00,3.73,1988
3,4,한희민,빙그레,16,5,5,31,20,188.0,4.93,2.63,0.77,0.293,78.20,3.11,4.11,3.43,3.55,3.00,1988
4,5,오영일,MBC,7,11,2,28,21,176.0,3.27,2.56,0.46,0.323,71.90,3.48,2.09,3.44,3.39,2.86,1988


In [11]:
pitcher_df_old.tail()

,#,선수명,팀명,승,패,세,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,year
447,51,성낙수,삼성,3,2,0,19,9,55.0,3.44,2.13,1.64,0.290,71.60,4.91,-0.63,4.71,4.65,-0.88,1983
448,52,엄평재,해태,0,2,0,13,9,11.0,9.82,13.09,3.27,0.270,59.10,16.36,-1.92,9.11,9.80,-0.88,1983
449,53,이선희,삼성,5,13,0,29,9,127.0,3.97,4.18,1.20,0.261,74.30,3.76,0.26,4.45,4.46,-0.89,1983
450,54,김덕열,롯데,0,1,0,9,5,21.7,3.74,3.74,2.49,0.288,72.80,6.23,-0.66,6.66,6.64,-0.92,1983
451,55,강철원,OB,4,2,0,17,6,62.7,3.02,3.30,1.72,0.288,68.10,6.32,-1.03,5.42,5.33,-1.26,1983


In [12]:
pitcher_df_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   #        452 non-null    object
 1   선수명      452 non-null    object
 2   팀명       452 non-null    object
 3   승        452 non-null    object
 4   패        452 non-null    object
 5   세        452 non-null    object
 6   경기       452 non-null    object
 7   선발       452 non-null    object
 8   이닝       452 non-null    object
 9   삼진/9     452 non-null    object
 10  볼넷/9     452 non-null    object
 11  홈런/9     452 non-null    object
 12  BABIP    452 non-null    object
 13  LOB%     452 non-null    object
 14  ERA      452 non-null    object
 15  RA9-WAR  452 non-null    object
 16  FIP      452 non-null    object
 17  kFIP     452 non-null    object
 18  WAR      452 non-null    object
 19  year     452 non-null    object
dtypes: object(20)
memory usage: 70.8+ KB


- 데이터프레임 저장

In [13]:
pitcher_df_old.to_csv('datas/pitcher_df_old.csv', encoding='utf-8')

In [14]:
pitcher_df_old = pd.read_csv('datas/pitcher_df_old.csv', index_col=0)

pitcher_df_old.head()

,#,선수명,팀명,승,패,세,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,year
0,1,선동열,해태,16,5,10,31,12,178.3,10.09,1.77,0.15,0.269,83.9,1.21,9.55,0.63,1.26,12.18,1988
1,2,윤학길,롯데,18,10,3,35,25,234.0,4.73,2.04,0.35,0.307,73.1,3.15,5.20,2.64,2.73,6.62,1988
2,3,최동원,롯데,7,3,3,16,4,83.3,8.96,2.59,0.43,0.317,80.9,2.05,2.51,1.48,2.00,3.73,1988
3,4,한희민,빙그레,16,5,5,31,20,188.0,4.93,2.63,0.77,0.293,78.2,3.11,4.11,3.43,3.55,3.00,1988
4,5,오영일,MBC,7,11,2,28,21,176.0,3.27,2.56,0.46,0.323,71.9,3.48,2.09,3.44,3.39,2.86,1988


#### 데이터 파싱 (2015 ~ 2020)

In [15]:
url = 'http://www.kbreport.com/leader/pitcher/main'

driver = webdriver.Chrome()

driver.get(url)

In [16]:
# 연도 선택 토글에 들어있는 연도
years = driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select").find_elements(By.TAG_NAME, "option")

years_list = [year.text.split('-')[1] for year in years]
years_list

['2024',
 '2023',
 '2022',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011']

- 필요한건 2015년 ~ 2020년이므로 슬라이싱

In [17]:
years_list = years_list[4:10]

years_list

['2020', '2019', '2018', '2017', '2016', '2015']

- years_list 반복문을 돌면서 데이터 파싱
- 1983 ~ 1988 데이터를 얻어올 때와는 다르게 선수가 많아 페이지를 넘어가야하는 것에 유의
- 또한 연도 선택 토글이 시작 시즌, 종료 시즌 2개로 되어있음

In [18]:
pitcher_data = []

for year in tqdm(years_list) :
    # 연도 선택 토글
    year_toggle_start = Select(driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select"))
    year_toggle_end = Select(driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select2"))
    year_toggle_start.select_by_value(str(year))
    year_toggle_end.select_by_value(str(year))

    # "결과" 선택 버튼
    result_btn = driver.find_element(By.CSS_SELECTOR, "#searchFrm > div.ltb-select-right > button")
    result_btn.click()
    sleep(0.1)

    # Page Row를 100개로 맞추기
    page_row = Select(driver.find_element(By.CSS_SELECTOR, "#resultListDiv > div.page-row-box > select"))
    page_row.select_by_value(str(100))
    sleep(0.5)

    # Page Num
    # 첫 페이지(<<), 이전 페이지(<), 다음 페이지(>), 마지막 페이지(>>) 버튼은 항상 고정이므로 총 page num에서 4개를 빼주기
    page_num = len(driver.find_element(By.CSS_SELECTOR, "#paging").find_elements(By.TAG_NAME, "a")) - 4

    # Page Num 넘기면서 선수정보 가져오기
    for num in range(page_num) :
        # 선수정보 가져오기
        pitcher_count = len(driver.find_element(By.CSS_SELECTOR, "#resultListDiv > table > tbody").find_elements(By.TAG_NAME, "tr"))

        for i in range(2, pitcher_count+1) :  # 가장 처음 tr 태그는 컬럼명이니 제외
            pitcher_info = driver.find_element(By.CSS_SELECTOR, f"#resultListDiv > table > tbody > tr:nth-child({i})")

            pitcher_info_list = pitcher_info.text.split()
            pitcher_info_list.append(year)
            pitcher_data.append(pitcher_info_list)

        print(pitcher_info_list)
        # 다음 페이지(>) 버튼 클릭
        driver.find_element(By.CSS_SELECTOR, "#paging > a.paging-num.next_page").click()
        sleep(1.0)

  0%|          | 0/6 [00:00<?, ?it/s]

['100', '박종기', '두산', '1', '2', '0', '0', '0', '8', '5', '26.0', '6.92', '4.85', '1.04', '0.338', '70.4', '5.54', '5.13', '5.16', '0.36', '0.43', '2020']
['200', '김재열', 'KIA', '0', '1', '0', '2', '0', '14', '0', '17.1', '6.75', '6.23', '1.56', '0.351', '65.1', '7.27', '6.73', '6.96', '-0.04', '-0.31', '2020']
['283', '이형범', '두산', '1', '2', '1', '1', '2', '27', '0', '25.2', '4.56', '5.61', '1.75', '0.294', '63.4', '7.71', '6.94', '7.27', '-0.89', '-0.31', '2020']


 17%|█▋        | 1/6 [00:09<00:46,  9.39s/it]

['100', '구승민', '롯데', '1', '4', '2', '6', '2', '41', '0', '36.0', '10.75', '5.25', '1.25', '0.377', '64.5', '6.25', '4.32', '3.98', '0.32', '-0.05', '2019']
['200', '최지광', '삼성', '3', '8', '2', '10', '1', '63', '0', '68.0', '7.94', '5.29', '0.53', '0.316', '73.5', '4.10', '4.25', '4.22', '-0.06', '0.76', '2019']
['257', '김기훈', 'KIA', '3', '6', '0', '0', '0', '19', '16', '79.1', '5.56', '7.37', '1.25', '0.228', '67.7', '5.56', '6.77', '7.07', '-1.13', '0.12', '2019']


 33%|███▎      | 2/6 [00:17<00:34,  8.72s/it]

...

 67%|██████▋   | 4/6 [00:34<00:17,  8.69s/it]

['100', '한승혁', 'KIA', '3', '2', '1', '9', '0', '36', '0', '33.1', '8.37', '5.13', '0.81', '0.351', '68.2', '4.86', '4.90', '4.77', '0.37', '0.80', '2016']
['200', '김성재', '롯데', '0', '0', '0', '0', '0', '6', '0', '3.2', '7.36', '4.91', '4.91', '0.333', '95.2', '7.36', '10.88', '10.88', '-0.06', '0.10', '2016']
['250', '한기주', 'KIA', '4', '3', '1', '1', '1', '29', '5', '56.2', '4.29', '4.76', '1.75', '0.350', '67.6', '7.62', '7.06', '7.43', '-0.92', '-0.19', '2016']


 83%|████████▎ | 5/6 [00:44<00:08,  8.94s/it]

['20', '정우람', 'SK', '7', '5', '16', '11', '5', '69', '0', '70.0', '11.57', '3.60', '0.39', '0.304', '72.9', '3.21', '2.63', '2.17', '2.59', '3.30', '2015']
['200', '임현준', '삼성', '0', '0', '0', '0', '0', '7', '0', '6.1', '8.53', '2.84', '4.26', '0.364', '34.1', '11.37', '8.83', '8.71', '-0.11', '-0.26', '2015']
['244', '송창식', '한화', '8', '7', '0', '11', '0', '64', '10', '109.0', '7.18', '5.28', '2.39', '0.279', '71.5', '6.44', '7.29', '7.42', '-0.86', '1.07', '2015']
['100', '장진용', 'LG', '1', '3', '0', '0', '0', '11', '9', '32.1', '5.57', '1.95', '1.39', '0.345', '60.4', '6.40', '5.50', '5.61', '0.28', '0.06', '2015']
['200', '임현준', '삼성', '0', '0', '0', '0', '0', '7', '0', '6.1', '8.53', '2.84', '4.26', '0.364', '34.1', '11.37', '8.83', '8.71', '-0.11', '-0.26', '2015']


100%|██████████| 6/6 [00:56<00:00,  9.45s/it]


In [19]:
len(pitcher_data)

1653

- pitcher_data를 value로 하는 데이터프레임 정의
- 컬럼 생성 (2015년도 기준)

In [20]:
# 컬럼 생성 (1983년도 기준)
columns = driver.find_element(By.CSS_SELECTOR, "#resultListDiv > table > tbody > tr:nth-child(1)")

report_columns = columns.text.split()

report_columns

['#',
 '선수명',
 '팀명',
 '승',
 '패',
 '세',
 '홀드',
 '블론',
 '경기',
 '선발',
 '이닝',
 '삼진/9',
 '볼넷/9',
 '홈런/9',
 'BABIP',
 'LOB%',
 'ERA',
 'FIP',
 'kFIP',
 'FIP-WAR',
 'RA9-WAR']

In [21]:
driver.close()

- report_columns에는 year 정보가 없으니 year 추가

In [22]:
report_columns.append('year')

- 2015 ~ 2020 투수 정보 데이터프레임 생성

In [23]:
pitcher_df_new = pd.DataFrame(data=pitcher_data, columns=report_columns)

pitcher_df_new.head()

,#,선수명,팀명,승,패,세,홀드,블론,경기,선발,...,볼넷/9,홈런/9,BABIP,LOB%,ERA,FIP,kFIP,FIP-WAR,RA9-WAR,year
0,1,스트레일리,롯데,15,4,0,0,0,31,31,...,2.36,0.46,0.278,75.5,2.50,3.01,2.68,8.28,9.43,2020
1,2,알칸타라,두산,20,2,0,0,0,31,31,...,1.36,0.54,0.289,79.0,2.54,3.10,2.84,8.16,9.76,2020
2,3,브룩스,KIA,11,4,0,0,0,23,23,...,1.43,0.24,0.302,75.5,2.50,2.72,2.48,6.53,7.47,2020
3,4,요키시,Hero,12,7,0,0,0,27,27,...,1.41,0.34,0.284,73.2,2.14,3.19,3.09,5.82,7.11,2020
4,5,플렉센,두산,8,4,0,0,0,21,21,...,2.31,0.46,0.307,71.3,3.01,2.73,2.31,5.58,4.66,2020


In [24]:
pitcher_df_new.tail()

,#,선수명,팀명,승,패,세,홀드,블론,경기,선발,...,볼넷/9,홈런/9,BABIP,LOB%,ERA,FIP,kFIP,FIP-WAR,RA9-WAR,year
1648,196,김수완,두산,0,0,0,0,0,9,2,...,5.28,2.35,0.375,66.9,8.80,7.66,7.80,-0.10,-0.08,2015
1649,197,심재민,KT,2,3,0,1,0,50,1,...,6.71,0.48,0.346,63.5,6.87,5.67,6.11,-0.10,-0.33,2015
1650,198,심규범,롯데,0,1,0,1,0,16,0,...,8.53,1.42,0.353,69.0,7.11,6.14,6.24,-0.10,-0.19,2015
1651,199,김현우,삼성,0,0,0,0,0,11,0,...,3.65,3.65,0.300,76.9,7.30,8.07,7.85,-0.11,-0.11,2015
1652,200,임현준,삼성,0,0,0,0,0,7,0,...,2.84,4.26,0.364,34.1,11.37,8.83,8.71,-0.11,-0.26,2015


In [25]:
pitcher_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653 entries, 0 to 1652
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   #        1653 non-null   object
 1   선수명      1653 non-null   object
 2   팀명       1653 non-null   object
 3   승        1653 non-null   object
 4   패        1653 non-null   object
 5   세        1653 non-null   object
 6   홀드       1653 non-null   object
 7   블론       1653 non-null   object
 8   경기       1653 non-null   object
 9   선발       1653 non-null   object
 10  이닝       1653 non-null   object
 11  삼진/9     1653 non-null   object
 12  볼넷/9     1653 non-null   object
 13  홈런/9     1653 non-null   object
 14  BABIP    1653 non-null   object
 15  LOB%     1653 non-null   object
 16  ERA      1653 non-null   object
 17  FIP      1653 non-null   object
 18  kFIP     1653 non-null   object
 19  FIP-WAR  1653 non-null   object
 20  RA9-WAR  1653 non-null   object
 21  year     1653 non-null   object
dtype

- 데이터프레임 저장

In [26]:
pitcher_df_new.to_csv('datas/pitcher_df_new.csv', encoding='utf-8')

In [44]:
pitcher_df_new = pd.read_csv('datas/pitcher_df_new.csv', index_col=0)

pitcher_df_new.head()

,#,선수명,팀명,승,패,세,홀드,블론,경기,선발,...,볼넷/9,홈런/9,BABIP,LOB%,ERA,FIP,kFIP,FIP-WAR,RA9-WAR,year
0,1,스트레일리,롯데,15,4,0,0,0,31,31,...,2.36,0.46,0.278,75.5,2.50,3.01,2.68,8.28,9.43,2020
1,2,알칸타라,두산,20,2,0,0,0,31,31,...,1.36,0.54,0.289,79.0,2.54,3.10,2.84,8.16,9.76,2020
2,3,브룩스,KIA,11,4,0,0,0,23,23,...,1.43,0.24,0.302,75.5,2.50,2.72,2.48,6.53,7.47,2020
3,4,요키시,Hero,12,7,0,0,0,27,27,...,1.41,0.34,0.284,73.2,2.14,3.19,3.09,5.82,7.11,2020
4,5,플렉센,두산,8,4,0,0,0,21,21,...,2.31,0.46,0.307,71.3,3.01,2.73,2.31,5.58,4.66,2020


## 연봉 정보 데이터
- statiz 사이트 활용
- 해당 사이트에서 선수 이름을 검색한 후 연봉(income)을 클릭하면 선수의 연봉 기록을 연도별로 확인 가능하다.
- 다만, 필요한 연도에 연봉 기록이 없기도 하며, 특히 옛날 선수의 연봉 기록은 거의 남아있지 않다.
- 일단 없는 연봉 기록을 NaN 처리하여 최대한 다 모아보자

### STATIZ
- url : https://statiz.sporki.com/

#### 데이터 파싱

In [28]:
url = 'https://statiz.sporki.com/'

driver = webdriver.Chrome()

driver.get(url)

- 선수 검색 버튼(돋보기) 클릭

In [29]:
search_btn = driver.find_element(By.CSS_SELECTOR, "body > div.warp > header > div.new_rightbox > div.new_searchPlayer")

search_btn.click()

- 선수 검색 팝업이 뜨면 선수 이름 검색

In [32]:
# 검색창 
search_input = driver.find_element(By.CSS_SELECTOR, "#s")
search_input.send_keys('최동원')

# 검색창에 선수이름이 입력되면 돋보기 버튼 클릭
search_btn_in = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.popBox.modal8 > form > div > input.btn")
search_btn_in.click()

- 검색 후 결과 페이지에서 "연봉" 버튼 클릭

In [33]:
income_btn = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.top_meum_box > div.team_smune > ul > li:nth-child(9) > a")
income_btn.click()

- 위 프로세스대로 현재 가지고 있는 모든 선수 이름을 검색하면 된다.
- 우선 pitcher_df_old, pitcher_df_new의 선수 이름을 모두 가져오기 위해 두 데이터프레임을 concat 해주자
- concat을 위해 두 데이터프레임의 컬럼을 비교하여 같게 맞춰주는 작업 필요

In [45]:
set(pitcher_df_new.columns) - set(pitcher_df_old.columns)

{'FIP-WAR', '블론', '홀드'}

- pitcher_df_new에 old 데이터프레임에는 없는 FIP-WAR, 홀드, 블론 컬럼이 있다.
- 두 컬럼을 확인해보자

In [47]:
pitcher_df_new[['FIP-WAR', '홀드', '블론']]

,FIP-WAR,홀드,블론
0,8.28,0,0
1,8.16,0,0
2,6.53,0,0
3,5.82,0,0
4,5.58,0,0
...,...,...,...
1648,-0.10,0,0
1649,-0.10,1,0
1650,-0.10,1,0
1651,-0.11,0,0


In [37]:
pitcher_df_new['홀드'].value_counts()

홀드
0     1025
1      166
2       89
3       63
4       44
5       36
6       27
11      26
9       26
7       26
10      20
8       18
12      13
13      12
14      11
15      10
16       8
17       7
25       4
20       3
22       3
24       3
18       3
19       2
21       2
31       1
33       1
27       1
40       1
23       1
37       1
Name: count, dtype: int64

In [38]:
pitcher_df_new['블론'].value_counts()

블론
0     1254
1      175
2       71
3       52
4       34
5       32
6       18
7        8
8        7
10       1
9        1
Name: count, dtype: int64

- 홀드, 블론 모두 0인 데이터가 월등히 많다.
    - 홀드 : 홀드(hd/hold)란, 세이브 규칙을 준수하여 해당 조건을 충족시키고 경기 도중 물러난 구원투수에게 홀드를 기록
    - 블론 : 세이브 조건에서 동점 혹은 역전을 허용한 경우 마운드에 있는 투수에게 주어지는 기록으로, 줄여서 '블론'이라고도 부른다
- 또한 FIP-WAR 값은 1980년대 데이터에서 구하기가 쉽지 않다.
- 세 컬럼 모두 없애주도록 하자

In [48]:
pitcher_df_new = pitcher_df_new.drop(['FIP-WAR', '홀드', '블론'], axis=1)

pitcher_df_new.columns

Index(['#', '선수명', '팀명', '승', '패', '세', '경기', '선발', '이닝', '삼진/9', '볼넷/9',
       '홈런/9', 'BABIP', 'LOB%', 'ERA', 'FIP', 'kFIP', 'RA9-WAR', 'year'],
      dtype='object')

- 두 데이터프레임의 컬럼이 같아졌으니 concat

In [49]:
pitcher_df = pd.concat([pitcher_df_old, pitcher_df_new])

pitcher_df.head()

,#,선수명,팀명,승,패,세,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,year
0,1,선동열,해태,16,5,10,31,12,178.3,10.09,1.77,0.15,0.269,83.9,1.21,9.55,0.63,1.26,12.18,1988
1,2,윤학길,롯데,18,10,3,35,25,234.0,4.73,2.04,0.35,0.307,73.1,3.15,5.20,2.64,2.73,6.62,1988
2,3,최동원,롯데,7,3,3,16,4,83.3,8.96,2.59,0.43,0.317,80.9,2.05,2.51,1.48,2.0,3.73,1988
3,4,한희민,빙그레,16,5,5,31,20,188.0,4.93,2.63,0.77,0.293,78.2,3.11,4.11,3.43,3.55,3.00,1988
4,5,오영일,MBC,7,11,2,28,21,176.0,3.27,2.56,0.46,0.323,71.9,3.48,2.09,3.44,3.39,2.86,1988


- 연도별로 동일 인물의 기록이 있어 이름 중복이 발생한다.
- 선수명을 가져올 때 unique로 고유값만 가져오도록 하자

In [53]:
pitcher_names = pitcher_df['선수명'].unique().tolist()

pitcher_names, len(pitcher_names)

(['선동열',
  '윤학길',
  '최동원',
  '한희민',
  '오영일',
  '김진욱',
  '이상군',
  '최일언',
  '김성길',
  '박동수',
  '양상문',
  '김훈기',
  '신완근',
  '김홍명',
  '김용수',
  '이상윤',
  '김용남',
  '이상구',
  '이용철',
  '문희수',
  '차동철',
  '윤석환',
  '조병천',
  '신동수',
  '이동석',
  '방수원',
  '김시진',
  '김신부',
  '한오종',
  '노상수',
  '김정수c',
  '김정행',
  '계형철',
  '장호연',
  '김대중',
  '손문곤',
  '오명록',
  '문병권',
  '천창호',
  '안창완',
  '박상범',
  '이문한',
  '김봉근',
  '김종석',
  '이길환',
  '진동한',
  '송유석',
  '홍성연',
  '예병준',
  '임재준',
  '유종겸',
  '하창우',
  '이현택',
  '이척기',
  '최진영',
  '박노준',
  '김대현',
  '김재열',
  '안성수',
  '이동수',
  '강만식',
  '임호균',
  '최창호',
  '지성규',
  '성준',
  '구명근',
  '서정용',
  '조도연',
  '강상진',
  '이충우',
  '김연철',
  '정윤수',
  '권영호',
  '김응국',
  '이상훈',
  '권기홍',
  '이국성',
  '장정순',
  '정용생',
  '박정현',
  '김강익',
  '배경환',
  '김일부',
  '양일환',
  '장태수',
  '박동경',
  '전용권',
  '황태환',
  '최상주',
  '한용덕',
  '박상열',
  '정삼흠',
  '김건우',
  '박철순',
  '정은배',
  '김태원',
  '김기태b',
  '이선희',
  '황기선',
  '정선두',
  '김경남',
  '김상기',
  '민문식',
  '김준희',
  '오문현',
  '정성만',
  '이진우',
  '하기룡',
  '박형열',
  '김일융',
  '성낙수',


- 이제 statiz 사이트에서 선수명을 검색하고 연봉 정보를 가져와보자
- 연봉 정보가 없는 경우 일단 NaN 처리

In [ ]:
income_data = []

for name in tqdm(pitcher_names) :
    # 선수 검색을 위한 돋보기 버튼 클릭
    search_btn = driver.find_element(By.CSS_SELECTOR, "body > div.warp > header > div.new_rightbox > div.new_searchPlayer")
    search_btn.click()

    # 돋보기 버튼 클릭 후 뜨는 검색창 팝업에 선수명 입력
    search_input = driver.find_element(By.CSS_SELECTOR, "#s")
    search_input.send_keys(name)

    # 검색창에 선수이름이 입력되면 돋보기 버튼 클릭
    search_btn_in = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.popBox.modal8 > form > div > input.btn")
    search_btn_in.click()

    # 연봉 버튼 클릭
    income_btn = driver.find_element(By.CSS_SELECTOR, "body > div.warp > div.container > section > div.top_meum_box > div.team_smune > ul > li:nth-child(9) > a")
    income_btn.click() 

    